needed to reload modules, otherwise you will have nan errors

In [10]:
#%load_ext autoreload
#%autoreload 2

### Data

In [11]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = [
    'IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6'
]
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (363, 18)
┌─────────┬────────┬────────┬───────┬───┬──────┬──────┬──────┬───────────────┐
│ target  ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ IRF7 ┆ IRF9 ┆ IRF6 ┆ unannotated_1 │
│ ---     ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---           │
│ str     ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64  ┆ f64  ┆ f64  ┆ i32           │
╞═════════╪════════╪════════╪═══════╪═══╪══════╪══════╪══════╪═══════════════╡
│ IL2     ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ IRF1    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ CCND1   ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ PRF1    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ IFNG    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 1.0  ┆ 0.0  ┆ 1             │
│ …       ┆ …      ┆ …      ┆ …     ┆ … ┆ …    ┆ …    ┆ …    ┆ …             │
│ ZC3HAV1 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ ZNF181  ┆ 0.0    ┆

In [12]:
from model.vega import VEGA
from model.decoder_default import DecoderVEGA
from model.encoder_with_mu_clamp import Encoder
from device import device

mask_np = mask.drop("target").to_numpy()

latent_dims= mask_np.shape[1]
input_dims = mask_np.shape[0]
dropout = 0.3
z_dropout = 0.5

vega = VEGA(
    encoder=Encoder(latent_dims, input_dims, dropout, z_dropout),
    decoder=DecoderVEGA(mask_np.T)
    ).to(device)

In [13]:
# model training
#vega, vega_losses, vega_klds, vega_mses = trainVEGA(vega, train, epochs = 20, beta = 0.0001) #takes about 2 mins on GPU # epoch 100

now use the common training functions

In [14]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,IRF2,STAT1,IRF8,STAT6,IRF4,IRF3,IRF5,STAT2,IRF7,IRF9,IRF6,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZC3HAV1""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [15]:
from training.training import run_vega_model
#run_vega_model(
#    vega=vega,
#    model_type="vega",
#    train_data=train,
#    valid_data=valid,
#    mask_df=mask,
#    epochs=5,
#    N=5
#)

bayes

In [16]:
from model.decoder_bayes import DecoderBayes

mask_np = mask.drop("target").to_numpy()

latent_dims= mask_np.shape[1]
input_dims = mask_np.shape[0]
dropout = 0.3
z_dropout = 0.5

vega_bayes = VEGA(
    encoder=Encoder(latent_dims, input_dims, dropout, z_dropout),
    decoder=DecoderBayes(mask_np.T)
    ).to(device)

In [17]:
from training.training import run_vega_model
run_vega_model(
    vega=vega_bayes,
    model_type="bayes",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=5,
    N=5
)

epoch:  0  train_loss:  88.55581   valid_loss:  68.31195
epoch:  0  train_loss:  50.475914   valid_loss:  38.785492
epoch:  0  train_loss:  43.56707   valid_loss:  33.424503
epoch:  0  train_loss:  40.885036   valid_loss:  31.99989
epoch:  0  train_loss:  39.394745   valid_loss:  30.909739


(VEGA(
   (decoder): DecoderBayes(
     (sparse_layer): BayesianSparseLayer()
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=363, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=17, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=17, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
            STAT5A    STAT5B     STAT4          IRF1         STAT3      IRF2  \
 IL2      0.001956  0.002147  0.000000  0.000000e+00  5.430671e-03 -0.000413   
 IRF1     0.015535 -0.001655  0.011112 -2.620842e-03 -9.450587e-04  0.005701   
 CCND1    0.001824  0.001388  0.000000 -1.387092e-03  9.261805e-04  0.000000   
 PR